In [6]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# 반경별 가중치
radii_weights = {
    500: 0.6,   # 500m 반경
    1000: 0.3,  # 1000m 반경
    2000: 0.1   # 2000m 반경
}

print("Imports & radii_weights loaded.")


Imports & radii_weights loaded.


In [7]:

def weighted_population_score(row, radii_weights, prefix="인구점수_"):
    """
    행(row)에서 인구점수_{반경}m 컬럼을 가중 평균.
    - 결측치(NaN)는 제외하고, 존재하는 반경만으로 가중치를 재정규화해서 평균.
    - 모두 결측일 경우 NaN 반환.
    """
    vals, ws = [], []
    for r, w in radii_weights.items():
        col = f"{prefix}{r}m"
        if col in row and pd.notna(row[col]):
            vals.append(float(row[col]))
            ws.append(float(w))
    if not vals:
        return np.nan
    ws = np.array(ws, dtype=float)
    ws /= ws.sum()  # 사용된 반경만 1로 재정규화
    return float(np.dot(np.array(vals, dtype=float), ws))

def zcdf(series):
    """Z-score 표준화 후 CDF(정규 누적분포) -> 0~100 점수로 변환"""
    s = pd.Series(series, dtype=float)
    mu = s.mean()
    sigma = s.std(ddof=0)
    if sigma == 0 or np.isnan(sigma):
        # 분산이 0이면 모두 동일 값 -> 50점
        return pd.Series(np.full(len(s), 50.0), index=s.index)
    z = (s - mu) / sigma
    return pd.Series(norm.cdf(z) * 100.0, index=s.index)

print("Helper functions ready.")


Helper functions ready.


In [8]:
hangang_path = "hangang_parks_rank.json"
nonhangang_path = "nonhangang_parks_full_table.json"

hangang_df = pd.read_json(hangang_path)
nonhangang_df = pd.read_json(nonhangang_path)

print(f"Loaded: hangang={hangang_df.shape}, nonhangang={nonhangang_df.shape}")
display(hangang_df.head(2))
display(nonhangang_df.head(2))


Loaded: hangang=(11, 12), nonhangang=(47, 14)


,공원명,도보점수,parking_score,구_list,인구점수_500m,인구점수_1000m,인구점수_2000m,총점_500m,총점_1000m,총점_2000m,한강_종합점수,rank
0,강서한강공원,0.000,30.08,[],NaN,NaN,NaN,6.016,11.5771,12.2168,9.936633,11
1,광나루한강공원,53.425,67.56,[],NaN,NaN,40.437,45.567,51.1281,53.6544,50.116500,9


,공원명,도보점수,parking_score,구_list,인구점수_500m,인구점수_1000m,인구점수_2000m,총점_500m,총점_1000m,총점_2000m,rank_500m,rank_1000m,rank_2000m,컴포넌트_종합(평균)
0,경의선숲길,93.377,0.00,[마포구],NaN,21.481,40.451,65.2399,60.3224,64.1164,14,15,12,38.82725
1,고척근린공원,63.576,35.96,[구로구],NaN,NaN,32.254,54.5513,50.0171,51.7884,32,31,35,43.93000


In [9]:
# 1) 인구점수 가중 평균 (결측치 제외 후 가중치 재정규화)
hangang_df["인구가중점수"] = hangang_df.apply(
    weighted_population_score, axis=1, radii_weights=radii_weights, prefix="인구점수_"
)

# 최소값 기준 보정 (NaN 처리) — 한강 데이터만 사용
min_val_hg = hangang_df["인구가중점수"].dropna().min()
# 모두 NaN인 극단 상황 대비
if pd.isna(min_val_hg):
    min_val_hg = 0.0
hangang_df["인구가중점수"] = hangang_df["인구가중점수"].fillna(min_val_hg * 0.5)

# 2) 0~100 정규화 (Z-CDF)
hangang_df["인구가중점수_std"] = zcdf(hangang_df["인구가중점수"])

# 3) 순위 (내림차순)
hangang_df = hangang_df.sort_values("인구가중점수_std", ascending=False).reset_index(drop=True)
hangang_df["rank_pop"] = np.arange(1, len(hangang_df) + 1)

# 4) 저장
out_hangang_csv = "hangang_population_weighted.csv"
hangang_df.to_csv(out_hangang_csv, index=False, encoding="utf-8-sig")

print("한강공원 인구 가중 점수 산출 & 저장 완료 →", out_hangang_csv)
display(hangang_df[["공원명","인구점수_500m","인구점수_1000m","인구점수_2000m",
                    "인구가중점수","인구가중점수_std","rank_pop"]].head(10))


한강공원 인구 가중 점수 산출 & 저장 완료 → hangang_population_weighted.csv


,공원명,인구점수_500m,인구점수_1000m,인구점수_2000m,인구가중점수,인구가중점수_std,rank_pop
0,난지한강공원,NaN,43.009,77.791,51.70450,98.324800,1
1,광나루한강공원,NaN,NaN,40.437,40.43700,86.499643,2
2,양화한강공원,NaN,NaN,32.704,32.70400,65.578573,3
3,잠실한강공원,NaN,23.736,57.034,32.06050,63.403788,4
4,잠원한강공원,NaN,31.875,24.272,29.97425,56.086480,5
5,이촌한강공원,NaN,NaN,29.304,29.30400,53.677493,6
6,여의도한강공원,NaN,NaN,26.318,26.31800,42.905577,7
7,반포한강공원,NaN,15.399,44.235,22.60800,30.306719,8
8,뚝섬한강공원,NaN,NaN,19.247,19.24700,20.589861,9
9,망원한강공원,NaN,NaN,17.868,17.86800,17.209199,10


In [10]:

non = nonhangang_df.copy()

# 1) 인구점수 가중 평균
non["인구가중점수"] = non.apply(
    weighted_population_score, axis=1, radii_weights=radii_weights, prefix="인구점수_"
)

# 최소값 기준 보정 (NaN 처리) — 비한강 데이터만 사용
min_val_non = non["인구가중점수"].dropna().min()
if pd.isna(min_val_non):
    min_val_non = 0.0
non["인구가중점수"] = non["인구가중점수"].fillna(min_val_non * 0.5)

# 2) 0~100 정규화 (Z-CDF)
non["인구가중점수_std"] = zcdf(non["인구가중점수"])

# 3) 공원 단위 순위
non_sorted = non.sort_values("인구가중점수_std", ascending=False).reset_index(drop=True)
non_sorted["rank_pop"] = np.arange(1, len(non_sorted) + 1)

# 4) 구 단위 집계
#    - 구_list가 여러 개인 경우 해당 모든 구에 동일 점수 부여 (explode)
#    - 구_list가 비어 있거나 NaN이면 Park_nm에서 '_' 앞 부분을 District로 추정 (없으면 '미상')
def infer_districts(row):
    gl = row.get("구_list", None)
    if isinstance(gl, list) and len(gl) > 0:
        return gl
    name = str(row.get("공원명", ""))
    if "_" in name:
        return [name.split("_")[0]]
    return ["미상"]

non["Districts"] = non.apply(infer_districts, axis=1)
non_exploded = non.explode("Districts", ignore_index=True)

# District별 평균 점수 (표준화 점수의 평균으로 집계)
district_mean = (
    non_exploded.groupby("Districts", dropna=False)["인구가중점수_std"]
    .mean()
    .reset_index()
    .rename(columns={"Districts": "District", "인구가중점수_std": "인구가중점수_std_mean"})
)

# 구 순위
district_mean = district_mean.sort_values("인구가중점수_std_mean", ascending=False).reset_index(drop=True)
district_mean["rank_pop_district"] = np.arange(1, len(district_mean) + 1)

# 5) 저장
out_non_park_csv = "nonhangang_population_weighted_park.csv"
out_non_district_csv = "nonhangang_population_weighted_district.csv"

cols_park = ["공원명","구_list","인구점수_500m","인구점수_1000m","인구점수_2000m",
             "인구가중점수","인구가중점수_std","rank_pop"]
non_sorted[cols_park].to_csv(out_non_park_csv, index=False, encoding="utf-8-sig")
district_mean.to_csv(out_non_district_csv, index=False, encoding="utf-8-sig")

print("비한강 공원 인구 가중 점수 산출 & 저장 완료 →",
      out_non_park_csv, ",", out_non_district_csv)

display(non_sorted[cols_park].head(10))
display(district_mean.head(10))


비한강 공원 인구 가중 점수 산출 & 저장 완료 → nonhangang_population_weighted_park.csv , nonhangang_population_weighted_district.csv


,공원명,구_list,인구점수_500m,인구점수_1000m,인구점수_2000m,인구가중점수,인구가중점수_std,rank_pop
0,서울숲,[성동구],100.000,100.000,38.562,93.85620,99.920571,1
1,용두근린공원(용두공원),[동대문구],78.901,86.282,83.808,81.60600,99.496436,2
2,어린이대공원,[광진구],98.250,26.500,52.549,72.15490,98.309512,3
3,파리근린공원(파리공원),[양천구],75.234,53.156,21.353,63.22250,95.506163,4
4,우장산근린공원(우장산/우장산근린공원),[강서구],NaN,45.991,100.000,59.49325,93.550232,5
5,아차산공원(아차산),[광진구],NaN,48.337,73.201,54.55300,90.013299,6
6,서울창포원,[도봉구],NaN,NaN,46.465,46.46500,81.495968,7
7,달맞이근린공원,[성동구],41.341,60.096,35.492,46.38260,81.390801,8
8,효창근린공원(효창공원),[용산구],50.796,36.771,33.308,44.83970,79.353757,9
9,매헌시민의숲,[서초구],NaN,35.488,68.339,43.70075,77.768636,10


,District,인구가중점수_std_mean,rank_pop_district
0,광진구,94.161406,1
1,성동구,90.655686,2
2,양천구,82.819403,3
3,용산구,79.353757,4
4,동대문구,68.362985,5
5,구로구,62.514550,6
6,강동구,58.143711,7
7,도봉구,54.903809,8
8,금천구,52.688338,9
9,서초구,45.342285,10
